In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, ZeroPadding2D, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
import h5py
import os
import cv2
import csv
import math
import tensorflow as tf
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
from keras.utils import Sequence

train_images_path = 'Z:/Smoke_DataSet2/Train/'
valid_images_path = 'Z:/Smoke_DataSet2/Valid/'
train_CSV_file = 'Z:/Smoke_DataSet2/Train.csv'
valid_CSV_file = 'Z:/Smoke_DataSet2/Valid.csv'
img_size = 224

class DataGenerator(Sequence):
    
    def __init__(self,csv_file,img_path,num,batch_size=25):
        
        self.csv_file = csv_file
        self.x = []
        self.y = np.zeros((int(num),4))
        self.batch_size = batch_size
        self.length = 0
        
        with open(self.csv_file,"r") as csv_data:
            csv_reader = csv.reader(csv_data, delimiter=',')
            line_no = 0
            
            for row in csv_reader:
                if line_no != 0:
                    
                    w = int(row[1])
                    h = int(row[2])
                    minx = abs(int(row[4]))
                    miny = abs(int(row[5]))
                    maxx = abs(int(row[6]))
                    maxy = abs(int(row[7]))
                    
                    self.y[line_no-1][0] = (minx/w)*img_size
                    self.y[line_no-1][1] = (miny/h)*img_size
                    self.y[line_no-1][2] = (maxx/w)*img_size
                    self.y[line_no-1][3] = (maxy/h)*img_size
                    
                    #self.y.append([minx,miny,maxx,maxy])
                    self.x.append(img_path + row[0])
                    line_no += 1
                    
                else:
                    line_no += 1
        self.length = (math.ceil(len(self.x) / self.batch_size))
        #print(len(self.x),self.length)
        #print(self.y)
                    
    def __loadimages(self,batch):
            
        img = []
            
        for filename in batch:
                
            im = cv2.resize(cv2.imread(filename), (img_size, img_size))
            img.append(im)
                
        return np.array(img)
            
    def __len__(self):
        return self.length
                
    def __getitem__(self,idx):
            
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
            
        images = self.__loadimages(batch_x).astype('float32')
            
        return images,batch_y

In [ ]:
train_datagen = DataGenerator(train_CSV_file,train_images_path,955,20)
valid_datagen = DataGenerator(valid_CSV_file,valid_images_path,100,10)

In [ ]:
#del Localizer
Localizer = Sequential()

Localizer.add(Conv2D(64,(5,5),input_shape=(224,224,3)))
Localizer.add(Conv2D(128,(3,3)))
Localizer.add(MaxPooling2D())
Localizer.add(Activation('relu'))
Localizer.add(Conv2D(256,(2,2)))
Localizer.add(Conv2D(256,(2,2)))
Localizer.add(MaxPooling2D())
Localizer.add(Activation('relu'))
Localizer.add(Conv2D(512,(2,2)))
Localizer.add(Conv2D(512,(2,2)))
Localizer.add(MaxPooling2D())
Localizer.add(Activation('relu'))
Localizer.add(Flatten())
Localizer.add(Dense(50,activation='relu'))
Localizer.add(Dense(4,activation='relu'))

Localizer.summary()

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint("SmokeLocalization-{loss:.3f}.h5", 
                             monitor="loss", 
                             verbose=1, 
                             save_best_only=True,
                             save_weights_only=True, 
                             mode="auto", 
                             period=1)

In [ ]:
Localizer.compile(loss=keras.losses.mean_squared_logarithmic_error, 
              optimizer=keras.optimizers.SGD(lr=0.01), 
              metrics=["accuracy"])

In [ ]:
history = Localizer.fit_generator(train_datagen, 
                    steps_per_epoch=48, 
                    epochs=5, 
                    validation_data=valid_datagen,
                    validation_steps=10,
                    callbacks = [checkpoint])

In [ ]:
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()